In [1]:
%pip install geopandas

  Obtaining dependency information for geopandas from https://files.pythonhosted.org/packages/3e/cf/ede993ed7070c5487e6db550aad490178c06eaa48ad26ff5e5263f995fba/geopandas-0.14.1-py3-none-any.whl.metadata
  Obtaining dependency information for fiona>=1.8.21 from https://files.pythonhosted.org/packages/a5/40/f25a33d54825d52f8b7f53adf8cb6d8f554179feb942bf2983c8b440decf/fiona-1.9.5-cp311-cp311-macosx_10_15_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 1.2 MB/s eta 0:00:00
  Obtaining dependency information for pyproj>=3.3.0 from https://files.pythonhosted.org/packages/84/a6/a300c1b14b2112e966e9f90b18f9c13b586bdcf417207cee913ae9005da3/pyproj-3.6.1-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for shapely>=1.8.0 from https://files.pythonhosted.org/packages/81/50/c7768a0a71c012464927228b6b949e55ad8d7ed50325b56b7224ea7dc7a6/shapely-2.0.2-cp311-cp311-macosx_10_9_x86_64.whl.metadata
  Obtaining dependency information for click~=8.

In [2]:
import geopandas as gpd
from shapely.geometry import Polygon
from pyproj import Proj, transform

In [3]:
gdf = gpd.read_file('TL_SCCO_SIG.shp', encoding='cp949')

In [4]:
gdf['sido'] = '서울'
gdf.rename(columns={
    "SIG_CD": "sig_code",
    "SIG_ENG_NM": "sig_eng_name",
    "SIG_KOR_NM": "sig_kor_name",
}, inplace=True)

gdf = gdf[["sig_code", "sido", "sig_eng_name", "sig_kor_name", "geometry"]]

In [5]:
gdf['sig_code'] = gdf['sig_code'].astype('int')
# SRID(CRS)를 설정한다
gdf = gpd.GeoDataFrame(gdf, crs='EPSG:4326', geometry=gdf['geometry'])

In [6]:
utm_k_srid = 5179
project_utm_k = Proj(init=f'epsg:{utm_k_srid}')
project_wgs84 = Proj(init='epsg:4326')


def utm_k_polygon_to_wgs84(polygon):
    """
    UTM-K 좌표계를 WGS84로 변환하여 반환한다
    
    :param polygon: utm-k 좌표계를 가진 Polygon 데이터 
    :return: WGS84 좌표계를 가진 Polygon 데이터
    """
   
    # 좌표계를 변환한다
    wgs84_coordinates = [transform(project_utm_k, project_wgs84, lon, lat) for lon, lat in polygon.exterior.coords]

    # longitude, latitude 순서를 변경하고 Polygon 객체를 생성한다
    wgs84_polygon = Polygon([(lat, lon) for lon, lat in wgs84_coordinates])

    return wgs84_polygon

/Users/kimjintae/Python/vPy311_cn-bis-preprocess/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/Users/kimjintae/Python/vPy311_cn-bis-preprocess/lib/python3.11/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [7]:
# utm-k 좌표계를 wgs84 좌표계로 변환한다
gdf['geometry'] = gdf['geometry'].apply(utm_k_polygon_to_wgs84)

/var/folders/kd/ln3s3sbd24g80yqyfgdk36180000gn/T/ipykernel_23892/558079026.py:15: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  wgs84_coordinates = [transform(project_utm_k, project_wgs84, lon, lat) for lon, lat in polygon.exterior.coords]


In [8]:
# geojson 파일로 저장한다
gdf.to_file("hang_jeong_gu.geojson", driver="GeoJSON")